# rhAmpSeq Clone Screening Report

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display, Markdown
from itables import init_notebook_mode, show, options
init_notebook_mode(all_interactive=True)
options.column_filters = "footer"

In [ ]:
df = pd.read_csv('stats.csv')
df["sample_aligned_read_num"] = df["total_aligned_read_num"].groupby(df["miseq_sample_name"]).transform("sum")
df = df[["plate", "well", "total_read_num", "sample_aligned_read_num", "aaanid", "beacon_placement_percentage","beacon_fidelity", "cargo_placement_percentage"]]

# predict monoallelic or biallelic
df["alleles"] = df["beacon_placement_percentage"].apply(lambda x: "No" if x <= 25 else ("biallelic" if x > 75 else "monoallelic"))
df["alleles"] = df[["beacon_placement_percentage", "cargo_placement_percentage"]].apply(lambda x: "No" if x.sum() <= 25 else ("biallelic" if x.sum() > 75 else "monoallelic"), axis=1)

In [ ]:
for plate in df["plate"].drop_duplicates().sort_values():
    d = df.loc[df["plate"] == plate].drop("plate", axis=1)
    d = d.pivot(index=["well", "total_read_num", "sample_aligned_read_num"], columns=["aaanid"]).dropna(axis='columns', how='all')
    d = d[["cargo_placement_percentage", "beacon_placement_percentage", "beacon_fidelity", "alleles"]].rename(columns={"beacon_placement_percentage": "BP%", "cargo_placement_percentage": "PGI %", "beacon_fidelity": "BF%"}).rename_axis(index={"total_read_num": "total #", "sample_aligned_read_num": "align #"})
    
    display(Markdown(f"## Plate {plate}"))
    show(d, footer=True)